In [1]:
from ultralytics import YOLO
import cv2
import math


# --- KONFIGURASI ---
url = "http://10.84.52.177:8080/video"
model = YOLO('yolov8n.pt')

# Menggunakan kamus yang sudah lengkap
KATEGORI_SAMPAH_LENGKAP = {
    'bottle': {'kategori_utama': 'Daur Ulang', 'sub_kategori': 'Plastik', 'contoh_jenis': 'PET, HDPE'},
    'cup': {'kategori_utama': 'Daur Ulang', 'sub_kategori': 'Plastik/Kertas', 'contoh_jenis': 'PP, Karton'},
    'apple': {'kategori_utama': 'Organik', 'sub_kategori': 'Sisa Makanan', 'contoh_jenis': 'Buah'},
    'banana': {'kategori_utama': 'Organik', 'sub_kategori': 'Sisa Makanan', 'contoh_jenis': 'Buah'},
    'cell phone': {'kategori_utama': 'B3', 'sub_kategori': 'Sampah Elektronik (E-Waste)', 'contoh_jenis': 'Ponsel'},
    'person': {'kategori_utama': 'Bukan Sampah', 'sub_kategori': 'Manusia', 'contoh_jenis': '-'}
}

WARNA_KATEGORI = {
    'Organik': (0, 128, 0), 'Daur Ulang': (255, 165, 0), 'Bukan Sampah': (0, 255, 255)
}

# --- PENGATURAN OPTIMISASI ---
RESIZE_WIDTH = 640
RESIZE_HEIGHT = 480
CONFIDENCE_THRESHOLD = 0.4

cap = cv2.VideoCapture(url)

if not cap.isOpened():
    print("Error: Tidak bisa membuka stream kamera.")
else:
    print("Kamera terhubung! Menjalankan deteksi dengan kategorisasi lengkap. Tekan 'q' untuk keluar.")
    
    while True:
        ret, frame = cap.read()
        if not ret: break

        frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))
        
        results = model(frame, stream=True, verbose=False, conf=CONFIDENCE_THRESHOLD)

        for r in results:
            for box in r.boxes:
                cls_name = model.names[int(box.cls[0])]
                
                if cls_name in KATEGORI_SAMPAH_LENGKAP:
                    info = KATEGORI_SAMPAH_LENGKAP[cls_name]
                    kategori_utama = info['kategori_utama']
                    sub_kategori = info['sub_kategori']
                    
                    warna = WARNA_KATEGORI.get(kategori_utama, (255, 255, 255))
                    
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = math.ceil(box.conf[0] * 100) / 100
                    
                    # Buat 2 baris label untuk informasi yang lebih kaya
                    label1 = f"Kategori: {kategori_utama} ({confidence})"
                    label2 = f"Jenis: {sub_kategori}"
                    
                    # Gambar kotak dan dua baris teks
                    cv2.rectangle(frame, (x1, y1), (x2, y2), warna, 2)
                    cv2.putText(frame, label1, (x1, y1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, warna, 2)
                    cv2.putText(frame, label2, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, warna, 2)

        cv2.imshow('Smart Bin - Klasifikasi Lengkap', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
print("Program dihentikan.")

Kamera terhubung! Menjalankan deteksi dengan kategorisasi lengkap. Tekan 'q' untuk keluar.
Program dihentikan.
